# Agrupación y Segmentación de Vecindarios en Toronto
## Parte 2

Explore y agrupe los vecindarios de Toronto. Puede decidir trabajar sólo con los municipios que contienen la palabra Toronto y luego replicar el mismo análisis que hicimos con los datos de la ciudad de Nueva York. Es su decisión.

Sólo asegúrese:

1.     Agregar suficientes celdas de Anotación para explicar lo que decidió hacer y para informar cualquier observación que haga.


Primero, importamos las librerías necesarias.
Después leemos los resultados del ejercicio anterior.

In [29]:
# Importamos "Numpy" como 'np'
import numpy as np
# Importamos "Pandas" como 'pd'
import pandas as pd 
# Importamos "Requests"
import requests
# Leemos el archivo del ejercicio anterior y lo asignamos a la variable "df_merged"
df_merged = pd.read_csv("Toronto_Q2.csv")

### Funciones predefinidas en Labs anteriores

In [35]:
# Función "getNearbyVenues"
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 50
    
    headers = {
                'Accept': 'application/json',
@hidden_cell    'Authorization': 'fsq3G9HJt7QaumR0S77rZWMifEKevD2/M+HJqwfYs1FEMbM='
    }

    URL = 'https://api.foursquare.com/v3/places/search?ll={},{}&radius={}&limit={}'

    venues_list = []
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)

        url = URL.format(lat, lng, radius, LIMIT)
        data = requests.get(url, headers=headers).json()

        # Procesamos venues_list
        for record in data['results']:
            venues_list.append({
                'Vecindario': name,
                'Latitud': lat,
                'Longitud': lng,
                'Venue': record['name'],
                'Venue Latitude': record['geocodes']['main']['latitude'],
                'Venue Longitude': record['geocodes']['main']['longitude'],
                'Venue Category': record['categories'][0]['name'],
            })

    # Creamos el dataFrame 'nearby_venues' con 'venues_list'
    nearby_venues = pd.DataFrame(venues_list)

    return(nearby_venues)


# Función que nos devuelve la categoría de cada sitio
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    

# Función quenos devuelve los lugares con más puntuación
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Importamos el resto de librerías que nos faltan:

In [31]:
# Importamos la librería JSON
import json 

# Instalamos geopy (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge geopy --yes 

# Importamos "Nominatim" desde "geopy.geocoders" para convertir una direción en coordenadas
from geopy.geocoders import Nominatim

# Importamos "Matplotlib.cm" como 'cm'
import matplotlib.cm as cm
# Importamos "Matplotlib.colors" como 'colors'
import matplotlib.colors as colors

# Importamos "KMeans" desde "sklearn.cluster"
from sklearn.cluster import KMeans

# Importamos "Folium" (En este caso ya está instalado, en caso contrario deshaz el comentario)
# c!onda install -c conda-forge folium=0.5.0 --yes

#Importamos "Folium"
import folium

print('✅  Todas las librerías se han importado correctamente ✅')

✅  Todas las librerías se han importado correctamente ✅


2.     Generar mapas para visualizar sus vecindarios y cómo se agrupan.

Dibujamos un mapa de Toronto con todos los Vecindarios:

In [32]:

#43.628841 	-79.520999
latitude = 43.628841
longitude = -79.520999


# create map of New York using latitude and longitude values
mapa_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, municipio, vecindario in zip(df_merged['Latitud'], df_merged['Longitud'], df_merged['Municipio'], df_merged['Vecindario']):
    label = '{}, {}'.format(vecindario, municipio)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(mapa_toronto)  
    
mapa_toronto



Cogemos las filas que contengan "Toronto" y usamos "getNearbyVenues"

In [33]:
datos = df_merged.set_index('Municipio').filter(regex=".*Toronto",axis=0)
datos.head()

,CódigoPostal,Vecindario,Latitud,Longitud
Municipio,,,,
Downtown Toronto,M5A,Regent Park / Harbourfront,43.654260,-79.360636
Downtown Toronto,M5B,"Garden District, Ryerson",43.657162,-79.378937
Downtown Toronto,M5C,St. James Town,43.651494,-79.375418
East Toronto,M4E,The Beaches,43.676357,-79.293031
Downtown Toronto,M5E,Berczy Park,43.644771,-79.373306


In [36]:
datos = df_merged.set_index('Municipio').filter(regex=".*Toronto",axis=0)
datos.head()


venuesToronto = getNearbyVenues(names = datos['Vecindario'],
                                   latitudes = datos['Latitud'],
                                   longitudes = datos['Longitud'])

Regent Park / Harbourfront
Garden District, Ryerson
St. James Town
The Beaches


IndexError: list index out of range

Una vez que esté satisfecho con su análisis, envíe un enlace al nuevo Notebook en su repositorio de Github. (3 puntos)